In [ ]:
# default_exp process

# Multithreading

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def multi_thread(fn, array_inputs, max_workers=None, desc="Multi-thread Pipeline", unit="Samples", verbose=True):
    from concurrent.futures import ThreadPoolExecutor
    from functools import partial
    from glob import glob
    from multiprocessing import Pool
    from tqdm import tqdm

    def _wraper(x):
        i, input = x
        return {i: fn(input)}

    array_inputs = [(i, _) for i, _ in enumerate(array_inputs)]
    if verbose:
        with tqdm(total=len(array_inputs), desc=desc, unit=unit) as progress_bar:
            outputs = {}
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                for result in executor.map(_wraper, array_inputs):
                    outputs.update(result)
                    progress_bar.update(1)
    else:
        outputs = {}
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            for result in executor.map(_wraper, array_inputs):
                outputs.update(result)
    if verbose:
        print('Finished')
    outputs = list(outputs.values())
    return outputs

## Example

In [1]:
# from avcv.process import multi_thread
show_doc(multi_thread)


# from glob import glob
# import mmcv
# import numpy as np
# from avcv.process import multi_thread
# from tqdm import tqdm


# def f(x):
#     mmcv.imread(x, channel_order='rgb')
#     return None

# paths = glob('../sample-imgs/*')
# inputs = np.random.choice(paths, 100)
# fast_imgs = multi_thread(f, inputs)

NameError: name 'show_doc' is not defined